# Belgium Bank Dataset Cleaning and Preparation Pipeline

## 📊 Project Overview

This notebook contains a professional data cleaning and preparation pipeline for the **Belgium Bank Dataset**, with the goal of transforming raw bank customer data into a structured, consistent, and analysis-ready format. The steps are fully documented for transparency and reproducibility.

---

## 🔑 Objectives:
- Load and inspect the raw dataset.
- Rename and translate columns to English.
- Detect and handle missing or invalid values.
- Standardize and normalize formats (e.g., IBAN, birth dates).
- Create a clean output dataset ready for analysis or machine learning.

---

## 🚀 Let's get started!


## ✅ Code Block 1: Importing Essential Libraries

In [26]:
# 📦 Importing required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# Optional: For regular expressions (Pandas uses regex internally but can import if needed)
import re


# ⚙️ Configuration for visualization
pd.set_option('display.max_columns', None)  # Show all columns in output
sns.set(style="whitegrid")

print("✅ Libraries imported successfully.")


✅ Libraries imported successfully.


## 📥 Step 1: Dataset Loading with Encoding Handling

In this step:
- We attempt to load the raw Belgium Bank dataset.
- Since datasets from Europe may contain special characters, we handle potential encoding issues.
- First, we try UTF-8 (standard), but if that fails, we fallback to Latin-1 encoding.

Below, we also preview the first 5 rows to confirm that data is loaded properly.


In [27]:
# 📥 Load dataset from raw_data folder with proper encoding handling
file_path = '../raw_data/Belgium-Iban.csv'  # Adjust path if necessary

# Attempt to read with utf-8, if it fails, fallback to 'latin-1'
try:
    df = pd.read_csv(file_path, encoding='utf-8')
    print("✅ Dataset loaded successfully with UTF-8 encoding.")
except UnicodeDecodeError:
    df = pd.read_csv(file_path, encoding='latin-1')
    print("⚠️ UTF-8 failed. Dataset loaded successfully using Latin-1 encoding.")

# 🧐 Preview first few rows to confirm data load
print("\n🔍 Preview of first 5 rows:")
print(df.head())




⚠️ UTF-8 failed. Dataset loaded successfully using Latin-1 encoding.

🔍 Preview of first 5 rows:
  geslacht   achternaam voorletter tussenvoegsel                straatnaam  \
0      Dhr       Jacobs         C.           NaN              Gasmeterlaan   
1      NaN        Dries         K.       van den              Groningenlei   
2        M    Vermeulen       KURT           NaN            Schongaustraat   
3        V     Oirschot         L.           van            Ertbrandstraat   
4     Mevr  Compernolle         R.           NaN  Albrecht rodenbachstraat   

  huisnummer toevoeging postcode            woonplaats  telefoonnummer  \
0        247        NaN     9000                  Gent     475644230.0   
1         15        NaN     2550               Kontich      32899040.0   
2          7        NaN     9100          Sint-niklaas      32966602.0   
3        187      bus 1     2950  Kapellen (antwerpen)      36052791.0   
4          4        NaN     8730               Beernem      5078

## 🏷️ Step 2: Rename Columns to English

To ensure our dataset is globally accessible and easily understandable by international stakeholders, we translate all column names from Dutch to English.

### Renamed Columns:
- gender
- last_name
- initials
- infix
- street_name
- house_number
- addition
- postal_code
- city
- phone_number
- date_of_birth
- account_number
- iban
- email
- bic

Below is the preview of the renamed dataset.


In [28]:
# 🏷️ Step 2: Renaming Columns to English and Standardized Format

# Mapping of Dutch to English column names
column_rename_map = {
    'geslacht': 'gender',
    'achternaam': 'last_name',
    'voorletter': 'initials',
    'tussenvoegsel': 'infix',
    'straatnaam': 'street_name',
    'huisnummer': 'house_number',
    'toevoeging': 'addition',
    'postcode': 'postal_code',
    'woonplaats': 'city',
    'telefoonnummer': 'phone_number',
    'geboortedatum': 'date_of_birth',
    'rekeningnummer': 'account_number',
    'iban': 'iban',
    'email': 'email',
    'bic': 'bic'
}

# Rename the columns
df.rename(columns=column_rename_map, inplace=True)

# ✅ Print renamed columns to verify
print("\n✅ Columns after renaming to English:")
print(df.columns.tolist())

# 🔍 Show first 5 rows to confirm changes
print("\n🔍 Preview of data after renaming columns:")
print(df.head())



✅ Columns after renaming to English:
['gender', 'last_name', 'initials', 'infix', 'street_name', 'house_number', 'addition', 'postal_code', 'city', 'phone_number', 'date_of_birth', 'account_number', 'iban', 'email', 'bic']

🔍 Preview of data after renaming columns:
  gender    last_name initials    infix               street_name  \
0    Dhr       Jacobs       C.      NaN              Gasmeterlaan   
1    NaN        Dries       K.  van den              Groningenlei   
2      M    Vermeulen     KURT      NaN            Schongaustraat   
3      V     Oirschot       L.      van            Ertbrandstraat   
4   Mevr  Compernolle       R.      NaN  Albrecht rodenbachstraat   

  house_number addition postal_code                  city  phone_number  \
0          247      NaN        9000                  Gent   475644230.0   
1           15      NaN        2550               Kontich    32899040.0   
2            7      NaN        9100          Sint-niklaas    32966602.0   
3          187    

## 🚀 Step 3: Data Types Standardization & Initial Cleaning
🎯 Goal for this Step:
Convert columns to proper data types (numbers, dates).
Standardize formats (remove spaces, fix IBAN, phone numbers).
Prepare data for deeper analysis.

### ✅ Code Block with Print Statements and Explanations:

In [29]:
# 📊 Step 3: Data Type Standardization and Initial Cleaning

print("\n✅ Starting data type standardization and cleaning process...\n")

# 🔢 Convert 'house_number' to string (since some houses have addition like '12A')
df['house_number'] = df['house_number'].astype(str)
print("🔹 Converted 'house_number' to string.")

# 🔢 Convert 'phone_number' and 'account_number' to string to preserve leading zeros and formatting
df['phone_number'] = df['phone_number'].astype(str).str.replace('.0', '', regex=False)
df['account_number'] = df['account_number'].astype(str).str.replace('.0', '', regex=False)
print("🔹 Converted 'phone_number' and 'account_number' to string and cleaned decimals.")

# 🧹 Clean 'iban': Remove all spaces for uniform formatting
df['iban'] = df['iban'].str.replace(' ', '').str.strip()
print("🔹 Cleaned IBAN numbers by removing spaces.")

# 🗓️ Standardize 'date_of_birth' to datetime format
df['date_of_birth'] = pd.to_datetime(df['date_of_birth'], errors='coerce', dayfirst=True)
print("🔹 Converted 'date_of_birth' to datetime format (NaT if invalid).")

# ✅ Clean 'postal_code': Make sure it's a string (some postal codes may start with zero)
df['postal_code'] = df['postal_code'].astype(str).str.strip()
print("🔹 Converted 'postal_code' to string and stripped whitespace.")

# 🧽 Strip leading/trailing whitespaces in textual columns (optional but professional)
text_columns = ['gender', 'last_name', 'initials', 'infix', 'street_name', 'addition', 'city', 'email', 'bic']
df[text_columns] = df[text_columns].apply(lambda x: x.str.strip() if x.dtype == "object" else x)
print("🔹 Stripped whitespace from all text columns.")

# ✅ Final DataFrame Overview
print("\n🔍 Data types after standardization:\n", df.dtypes)
print("\n🔍 Preview of cleaned data:\n", df.head())



✅ Starting data type standardization and cleaning process...

🔹 Converted 'house_number' to string.
🔹 Converted 'phone_number' and 'account_number' to string and cleaned decimals.
🔹 Cleaned IBAN numbers by removing spaces.
🔹 Converted 'date_of_birth' to datetime format (NaT if invalid).
🔹 Converted 'postal_code' to string and stripped whitespace.
🔹 Stripped whitespace from all text columns.

🔍 Data types after standardization:
 gender                    object
last_name                 object
initials                  object
infix                     object
street_name               object
house_number              object
addition                  object
postal_code               object
city                      object
phone_number              object
date_of_birth     datetime64[ns]
account_number            object
iban                      object
email                     object
bic                       object
dtype: object

🔍 Preview of cleaned data:
   gender    last_name initial

### ✅ Step 4 - Full Notebook Code & Markdown Example:
📦 Code Block 1: Identifying Missing Data

In [30]:
# 📊 Checking for missing values in each column
print("🔎 Checking missing values per column:\n")
missing_values = df.isnull().sum()
print(missing_values)

# 📏 Also showing percentage of missingness for stakeholders
print("\n📊 Percentage of missing values per column:\n")
missing_percent = (df.isnull().sum() / len(df)) * 100
print(missing_percent.round(2))


🔎 Checking missing values per column:

gender             139
last_name            4
initials            46
infix             6283
street_name          0
house_number         0
addition          6264
postal_code          0
city                 0
phone_number         0
date_of_birth     2995
account_number       0
iban                 0
email                0
bic                886
dtype: int64

📊 Percentage of missing values per column:

gender             1.87
last_name          0.05
initials           0.62
infix             84.59
street_name        0.00
house_number       0.00
addition          84.33
postal_code        0.00
city               0.00
phone_number       0.00
date_of_birth     40.32
account_number     0.00
iban               0.00
email              0.00
bic               11.93
dtype: float64


## ✅ Step 4: Handling Missing Data and Saving Clean & Garbage Files

- **No rows or columns were dropped** — all missing values were replaced with empty strings ('').
- **Two versions** of the dataset have been saved:
  - **Garbage Data File** (contains blanks but retains all raw records): `belgium_bank_garbage_data.csv`
  - **Clean Data File** (ready for analysis with blanks instead of NaNs): `belgium_bank_clean_data.csv`
- This ensures we keep **full transparency and flexibility** for future review and decision-making.



In [31]:
# 🛠️ Step 4: Handling Missing Data & Saving Both Garbage and Clean Files

# ✅ Replacing all missing values with empty strings to preserve all records
df.fillna('', inplace=True)

# ✅ Confirming that missing values are handled
print("\n🔄 Verifying that all missing values have been filled with blanks (should show 0 for all):\n")
print(df.isnull().sum())

# 💾 Save the "garbage data" version (with blanks but no removed data)
garbage_file_path = '../output/belgium_bank_garbage_data.csv'
df.to_csv(garbage_file_path, index=False, encoding='utf-8-sig')
print(f"\n💾 Garbage dataset saved successfully at: {garbage_file_path}")

# 💾 Save the "clean data" version for analysis
clean_file_path = '../output/belgium_bank_clean_data.csv'
df.to_csv(clean_file_path, index=False, encoding='utf-8-sig')
print(f"💾 Clean dataset saved successfully at: {clean_file_path}")



🔄 Verifying that all missing values have been filled with blanks (should show 0 for all):

gender               0
last_name            0
initials             0
infix                0
street_name          0
house_number         0
addition             0
postal_code          0
city                 0
phone_number         0
date_of_birth     2995
account_number       0
iban                 0
email                0
bic                  0
dtype: int64

💾 Garbage dataset saved successfully at: ../output/belgium_bank_garbage_data.csv
💾 Clean dataset saved successfully at: ../output/belgium_bank_clean_data.csv


## Step 5: Data Consistency & Validation

In this step, we will focus on **validating critical fields** such as IBAN, BIC, phone numbers, and dates of birth to ensure they follow expected formats. This is essential to maintain high-quality, reliable datasets for analysis and downstream use.

### ✅ Fields to Validate:
1. **IBAN Numbers** - Correct Belgian IBAN format.
2. **Phone Numbers** - Expected length and numeric format.
3. **Date of Birth** - Detect invalid/missing dates.
4. **BIC Codes** - Proper format and length (typically 8 or 11 characters).
5. **Duplicate Checks** - Avoid double entries (e.g., same IBAN or email).

Let's start!


In [32]:
# ✅ Cleaned dataset loaded for validation
print("✅ Cleaned dataset loaded for validation. Number of records:", len(clean_data))
print(clean_data.head(3))  # Quick peek

# ------------------- IBAN Validation ------------------- #
# IBAN must start with 'BE' followed by 14 digits
invalid_iban = clean_data[~clean_data['iban'].str.match(r'^BE\d{14}$')]
print(f"⚠️ Found {len(invalid_iban)} invalid IBAN(s).")
print(invalid_iban[['iban']].head(5))
invalid_iban.to_csv('../output/belgium_bank_invalid_iban.csv', index=False)
print("💾 Invalid IBANs saved to 'output/belgium_bank_invalid_iban.csv'.")

# ------------------- Phone Number Validation ------------------- #
# Ensure column is string and fill blanks
clean_data['phone_number'] = clean_data['phone_number'].fillna('').astype(str)

# Check for numeric-only, 8-10 digits
invalid_phone = clean_data[~clean_data['phone_number'].str.match(r'^\d{8,10}$')]
print(f"⚠️ Found {len(invalid_phone)} invalid phone number(s).")
print(invalid_phone[['phone_number']].head(5))
invalid_phone.to_csv('../output/belgium_bank_invalid_phone.csv', index=False)
print("💾 Invalid phone numbers saved to 'output/belgium_bank_invalid_phone.csv'.")

# ------------------- BIC Validation ------------------- #
# BIC must be 8 or 11 alphanumeric characters
clean_data['bic'] = clean_data['bic'].fillna('').astype(str)  # Ensure string type
invalid_bic = clean_data[
    (clean_data['bic'] != '') &
    (~clean_data['bic'].str.match(r'^[A-Z0-9]{8}([A-Z0-9]{3})?$'))
]
print(f"⚠️ Found {len(invalid_bic)} invalid BIC(s).")
print(invalid_bic[['bic']].head(5))
invalid_bic.to_csv('../output/belgium_bank_invalid_bic.csv', index=False)
print("💾 Invalid BICs saved to 'output/belgium_bank_invalid_bic.csv'.")



✅ Cleaned dataset loaded for validation. Number of records: 24
    gender last_name initials infix              street_name house_number  \
122    Dhr   Petegem        G   Van         Kunnenbergstraat            8   
309   Mevr   Verwimp        F        P.Van Langendonckstraat           19   
560      V     Melis       A.               Winterslagstraat          105   

    addition postal_code        city phone_number date_of_birth  \
122                 9660      Brakel    495211164    1973-12-17   
309                 9050  Gentbrugge     92314605                 
560    Bus 1        3600     Centrum    486405695    1966-09-20   

       account_number              iban                        email  \
122  780581655863      BE40780581655863      adarma.lieve@telenet.be   
309    1105666826      BE59001105666826  florette.verwimp@telenet.be   
560    1645777172      BE38001645777172         Melisanita@gmail.com   

          bic validation_issues  
122  GKCCBEBB                OK  
30

### ✅ Fix Phone and BIC Issues + Re-run Validations:

In [33]:
# ✅ Fix phone number: remove decimals and spaces
clean_data['phone_number'] = clean_data['phone_number'].fillna('').astype(str).str.replace(r'\.0$', '', regex=True).str.strip()

# ✅ Fix BIC: remove spaces
clean_data['bic'] = clean_data['bic'].fillna('').astype(str).str.replace(' ', '').str.strip()

# ------------------- Re-run Phone Number Validation ------------------- #
invalid_phone = clean_data[~clean_data['phone_number'].str.match(r'^\d{8,10}$')]
print(f"⚠️ Found {len(invalid_phone)} invalid phone number(s) AFTER FIX.")
print(invalid_phone[['phone_number']].head(5))
invalid_phone.to_csv('../output/belgium_bank_invalid_phone.csv', index=False)
print("💾 Invalid phone numbers saved to 'output/belgium_bank_invalid_phone.csv'.")

# ------------------- Re-run BIC Validation ------------------- #
invalid_bic = clean_data[
    (clean_data['bic'] != '') &
    (~clean_data['bic'].str.match(r'^[A-Z0-9]{8}([A-Z0-9]{3})?$'))
]
print(f"⚠️ Found {len(invalid_bic)} invalid BIC(s) AFTER FIX.")
print(invalid_bic[['bic']].head(5))
invalid_bic.to_csv('../output/belgium_bank_invalid_bic.csv', index=False)
print("💾 Invalid BICs saved to 'output/belgium_bank_invalid_bic.csv'.")


⚠️ Found 0 invalid phone number(s) AFTER FIX.
Empty DataFrame
Columns: [phone_number]
Index: []
💾 Invalid phone numbers saved to 'output/belgium_bank_invalid_phone.csv'.
⚠️ Found 0 invalid BIC(s) AFTER FIX.
Empty DataFrame
Columns: [bic]
Index: []
💾 Invalid BICs saved to 'output/belgium_bank_invalid_bic.csv'.


# 📊 Step 5: Final Validation & Correction of Belgium Bank Dataset
In this step, we perform **final checks** and corrections on:
- **Phone Numbers**: Ensuring correct numeric format, length.
- **BIC Codes**: Proper format, length, uppercase without spaces.

We'll also re-validate for any remaining issues and save final datasets.

**Deliverables:**
- `belgium_bank_clean_data_final.csv`: Final cleaned data
- `belgium_bank_invalid_phone.csv`: Remaining invalid phone numbers (if any)
- `belgium_bank_invalid_bic.csv`: Remaining invalid BIC codes (if any)

In [34]:
# ✅ Ensure phone_number is string and NaNs are empty strings
clean_data['phone_number'] = clean_data['phone_number'].fillna('').astype(str)

# ✅ Remove non-digit characters
clean_data['phone_number'] = clean_data['phone_number'].str.replace(r'\D', '', regex=True)



# Validate email format using regex
email_pattern = r'^[\w\.-]+@[\w\.-]+\.\w+$'
invalid_emails = clean_data[~clean_data['email'].str.match(email_pattern, na=False)]

print(f"⚠️ Found {len(invalid_emails)} invalid email(s).")
print(invalid_emails[['email']].head())

# Optionally save for review
invalid_emails.to_csv('../output/belgium_bank_invalid_emails.csv', index=False)
print("💾 Invalid emails saved to 'output/belgium_bank_invalid_emails.csv'.")





# ✅ Now safely apply regex check for valid 8-10 digit phone numbers
invalid_phone = clean_data[~clean_data['phone_number'].str.match(r'^\d{8,10}$', na=False)]

print(f"⚠️ Found {len(invalid_phone)} invalid phone number(s) AFTER FIX.")
print(invalid_phone[['phone_number']].head())

# 💾 Save invalid phone numbers
invalid_phone.to_csv('../output/belgium_bank_invalid_phone.csv', index=False)
print("💾 Invalid phone numbers saved to 'output/belgium_bank_invalid_phone.csv'.")

# ✅ Fix BICs: ensure string type, fill NaNs, remove spaces, and uppercase
clean_data['bic'] = clean_data['bic'].fillna('').astype(str).str.replace(r'\s+', '', regex=True).str.upper()

# ✅ Validate BICs (8 or 11 alphanumeric characters)
invalid_bic = clean_data[
    (clean_data['bic'] != '') &  # Not empty
    (~clean_data['bic'].str.match(r'^[A-Z0-9]{8}([A-Z0-9]{3})?$', na=False))
]

print(f"⚠️ Found {len(invalid_bic)} invalid BIC(s) AFTER FIX.")
print(invalid_bic[['bic']].head())

# 💾 Save invalid BICs
invalid_bic.to_csv('../output/belgium_bank_invalid_bic.csv', index=False)
print("💾 Invalid BICs saved to 'output/belgium_bank_invalid_bic.csv'.")

# 📂 Load Invalid IBAN File
invalid_iban_path = '../output/belgium_bank_invalid_iban.csv'
invalid_iban = pd.read_csv(invalid_iban_path)

# Display summary
print(f"⚠️ Total invalid IBANs detected: {len(invalid_iban)}")
print("🔍 Sample of invalid IBANs:")
display(invalid_iban.head(10))  # Display top 10 as sample




⚠️ Found 0 invalid email(s).
Empty DataFrame
Columns: [email]
Index: []
💾 Invalid emails saved to 'output/belgium_bank_invalid_emails.csv'.
⚠️ Found 0 invalid phone number(s) AFTER FIX.
Empty DataFrame
Columns: [phone_number]
Index: []
💾 Invalid phone numbers saved to 'output/belgium_bank_invalid_phone.csv'.
⚠️ Found 0 invalid BIC(s) AFTER FIX.
Empty DataFrame
Columns: [bic]
Index: []
💾 Invalid BICs saved to 'output/belgium_bank_invalid_bic.csv'.
⚠️ Total invalid IBANs detected: 0
🔍 Sample of invalid IBANs:


,gender,last_name,initials,infix,street_name,house_number,addition,postal_code,city,phone_number,date_of_birth,account_number,iban,email,bic,validation_issues


# 📊 Step 5: Full Dataset Inspection & Final Cleaning Actions

In this step, we will:

1. Inspect data types and value distributions to ensure everything looks consistent.
2. Perform validation and cleaning for **email addresses**.
3. Perform validation and cleaning for **names** (first and last names, initials).
4. Check and handle **duplicate rows** based on critical identifiers.
5. Export the fully cleaned dataset for analysis.

This step ensures that we **finalize the dataset quality** before moving into analysis and modeling phases.


In [35]:
# 🔍 Inspect basic information
print("✅ Dataset Preview:")
display(clean_data.head(5))

print("\n✅ Data Types:")
print(clean_data.dtypes)

print("\n🔢 Dataset shape (rows, columns):", clean_data.shape)

print("\n📊 Checking null counts per column:")
print(clean_data.isnull().sum())

print("\n📊 Unique counts in each column:")
print(clean_data.nunique())

print("\n🎯 Checking summary statistics for numeric-looking fields:")
print(clean_data.describe(include='all').transpose())


✅ Dataset Preview:


,gender,last_name,initials,infix,street_name,house_number,addition,postal_code,city,phone_number,date_of_birth,account_number,iban,email,bic,validation_issues
122,Dhr,Petegem,G,Van,Kunnenbergstraat,8,,9660,Brakel,495211164,1973-12-17,780581655863,BE40780581655863,adarma.lieve@telenet.be,GKCCBEBB,OK
309,Mevr,Verwimp,F,,P.Van Langendonckstraat,19,,9050,Gentbrugge,92314605,,1105666826,BE59001105666826,florette.verwimp@telenet.be,GEBABEBB,OK
560,V,Melis,A.,,Winterslagstraat,105,Bus 1,3600,Centrum,486405695,1966-09-20,1645777172,BE38001645777172,Melisanita@gmail.com,GEBABEBB,OK
1568,Dhr,Bals,P,,Vossekotstraat,62,,9100,St-Niklaas,478234393,1971-01-16,293056336140,BE02293056336140,bals_peter@telenet.be,GEBABEBB,OK
1675,Mevr,Boutens,R,,Kortrijkessteenweg,84,,9800,Denzie,499159550,,63450274439,BE13063450274439,BT.rosita@hotmail.be,GKCCBEBB,OK



✅ Data Types:
gender               object
last_name            object
initials             object
infix                object
street_name          object
house_number         object
addition             object
postal_code          object
city                 object
phone_number         object
date_of_birth        object
account_number       object
iban                 object
email                object
bic                  object
validation_issues    object
dtype: object

🔢 Dataset shape (rows, columns): (24, 16)

📊 Checking null counts per column:
gender               0
last_name            0
initials             0
infix                0
street_name          0
house_number         0
addition             0
postal_code          0
city                 0
phone_number         0
date_of_birth        0
account_number       0
iban                 0
email                0
bic                  0
validation_issues    0
dtype: int64

📊 Unique counts in each column:
gender                5
last_n

### ✅ Final Code Block with Markdown for Names Normalization

In [36]:
# ---
# ## ✨ Step 1: Standardizing Names (Last Names, Initials, Infix)
# - Title-case all last names and infix (e.g., "van" to "Van")
# - Upper-case initials (e.g., "k." to "K.")
# - Strip extra spaces and handle blanks
# ---

import pandas as pd

# ✅ Load the working clean dataset
file_path = r'C:\Users\Andre\OneDrive\Documents\Belgium-Bank-Dataset\output\belgium_bank_clean_data.csv'
df = pd.read_csv(file_path, dtype=str)
print("✅ Dataset loaded. Shape:", df.shape)

# ✅ Fill NaNs as blanks for consistency
df.fillna('', inplace=True)

# ✅ Strip and format names properly
df['last_name'] = df['last_name'].str.strip().str.title()  # Title case, strip spaces
df['initials'] = df['initials'].str.strip().str.upper()   # Upper case, strip spaces
df['infix'] = df['infix'].str.strip().str.title()         # Title case, strip spaces

# ✅ Preview to confirm the transformation
print("\n🔍 Preview after name standardization:")
print(df[['last_name', 'initials', 'infix']].head(10))

# ✅ Save updated dataset to a new clean file
output_path = r'C:\Users\Andre\OneDrive\Documents\Belgium-Bank-Dataset\output\belgium_bank_clean_data_with_fixed_names.csv'
df.to_csv(output_path, index=False)
print(f"\n💾 Cleaned dataset with fixed names saved to: {output_path}")

# ---
# ✅ Summary:
# - Names properly formatted
# - Dataset ready for next steps (email/IBAN/BIC/duplicates)
# ---


✅ Dataset loaded. Shape: (7428, 15)

🔍 Preview after name standardization:
     last_name initials    infix
0       Jacobs       C.         
1        Dries       K.  Van Den
2    Vermeulen     KURT         
3     Oirschot       L.      Van
4  Compernolle       R.         
5      Taalman       M.         
6  Blauwblomme       V.         
7  El-Khattabi       R.         
8       Jannis       Y.         
9     Vlaminck       C.         

💾 Cleaned dataset with fixed names saved to: C:\Users\Andre\OneDrive\Documents\Belgium-Bank-Dataset\output\belgium_bank_clean_data_with_fixed_names.csv


### Final Corrected Approach for Email Handling

In [37]:
# 📌 Step 2: Email Validation (Retain all rows, separate invalid emails)


# 🔽 Load dataset with fixed names
file_path = r'C:\Users\Andre\OneDrive\Documents\Belgium-Bank-Dataset\output\belgium_bank_clean_data_with_fixed_names.csv'
df = pd.read_csv(file_path, dtype=str)
print(f"✅ Dataset loaded. Shape: {df.shape}")

# 🧽 Fill blanks and strip whitespaces for safety
df['email'] = df['email'].fillna('').str.strip()

# ✅ Email validation pattern
email_pattern = r"^[\w\.-]+@[\w\.-]+\.\w+$"

# 🔍 Detect invalid emails
invalid_email_mask = ~df['email'].str.match(email_pattern, na=False)
invalid_emails_df = df[invalid_email_mask].copy()
print(f"⚠️ Found {len(invalid_emails_df)} invalid email(s).")

# 💾 Save rows with invalid emails to garbage file for review
garbage_output_path = r'C:\Users\Andre\OneDrive\Documents\Belgium-Bank-Dataset\output\belgium_bank_garbage_data.csv'
invalid_emails_df.to_csv(garbage_output_path, index=False)
print(f"💾 Invalid email rows saved to: {garbage_output_path}")

# 🧹 Blank out invalid emails in main dataset (retain row)
df.loc[invalid_email_mask, 'email'] = ''

# 💾 Save updated clean dataset (emails cleaned)
clean_emails_path = r'C:\Users\Andre\OneDrive\Documents\Belgium-Bank-Dataset\output\belgium_bank_clean_data_emails_valid.csv'
df.to_csv(clean_emails_path, index=False)
print(f"💾 Updated clean dataset saved to: {clean_emails_path}")

# ✅ Summary
print("\n📊 Email Cleaning Summary:")
print(f"Total records retained: {len(df)}")
print(f"Invalid emails moved to garbage file: {len(invalid_emails_df)}")
print("🚀 Ready for next cleaning step (phone, BIC, IBAN)!")


✅ Dataset loaded. Shape: (7428, 15)
⚠️ Found 8 invalid email(s).
💾 Invalid email rows saved to: C:\Users\Andre\OneDrive\Documents\Belgium-Bank-Dataset\output\belgium_bank_garbage_data.csv
💾 Updated clean dataset saved to: C:\Users\Andre\OneDrive\Documents\Belgium-Bank-Dataset\output\belgium_bank_clean_data_emails_valid.csv

📊 Email Cleaning Summary:
Total records retained: 7428
Invalid emails moved to garbage file: 8
🚀 Ready for next cleaning step (phone, BIC, IBAN)!


# ---
# ## 📞 Phone Number Validation & Cleanup
# ---

In [39]:
# ---
# ## 📞 Phone Number Validation & Cleanup
# ---

import pandas as pd

# ✅ Load the current working clean dataset
file_path = r'C:\Users\Andre\OneDrive\Documents\Belgium-Bank-Dataset\output\belgium_bank_clean_data_emails_valid.csv'
df = pd.read_csv(file_path, dtype=str)
print("✅ Dataset loaded. Shape:", df.shape)

# ✅ Define phone number validation pattern (8 to 10 digits only)
phone_pattern = r"^\d{8,10}$"

# ========================
# STEP 1: Find Invalid Phones
# ========================
invalid_phone_mask = ~df['phone_number'].str.match(phone_pattern, na=False)
invalid_phones = df[invalid_phone_mask].copy()

print(f"⚠️ Found {len(invalid_phones)} invalid phone number(s).")
print(invalid_phones[['phone_number']].head())

# ========================
# STEP 2: Add Invalid Phones to Garbage File
# ========================
# Load existing garbage file to append
garbage_file_path = r'C:\Users\Andre\OneDrive\Documents\Belgium-Bank-Dataset\output\belgium_bank_garbage_data.csv'
try:
    garbage_df = pd.read_csv(garbage_file_path, dtype=str)
except FileNotFoundError:
    garbage_df = pd.DataFrame()  # Create empty DataFrame if doesn't exist yet

# Append new invalid phones
updated_garbage_df = pd.concat([garbage_df, invalid_phones], ignore_index=True)

# Save updated garbage
updated_garbage_df.to_csv(garbage_file_path, index=False)
print(f"💾 Invalid phone numbers added to: {garbage_file_path}")

# ========================
# STEP 3: Blank Invalid Phone Numbers in Clean Dataset
# ========================
df.loc[invalid_phone_mask, 'phone_number'] = ''

# ========================
# STEP 4: Save Updated Clean Dataset
# ========================
clean_output_path = r'C:\Users\Andre\OneDrive\Documents\Belgium-Bank-Dataset\output\belgium_bank_clean_data_phones_valid.csv'
df.to_csv(clean_output_path, index=False)
print(f"💾 Updated clean dataset saved to: {clean_output_path}")

# ========================
# ✅ STEP 5: Summary
# ========================
print("\n📊 Phone Number Cleaning Summary:")
print("Total records retained:", len(df))
print("Invalid phone numbers moved to garbage file:", len(invalid_phones))
print("🚀 Ready for next cleaning step (BIC or IBAN)!")


✅ Dataset loaded. Shape: (7428, 15)
⚠️ Found 11 invalid phone number(s).
     phone_number
144           NaN
335   5.37014e+17
1581            3
2820          NaN
2821          NaN
💾 Invalid phone numbers added to: C:\Users\Andre\OneDrive\Documents\Belgium-Bank-Dataset\output\belgium_bank_garbage_data.csv
💾 Updated clean dataset saved to: C:\Users\Andre\OneDrive\Documents\Belgium-Bank-Dataset\output\belgium_bank_clean_data_phones_valid.csv

📊 Phone Number Cleaning Summary:
Total records retained: 7428
Invalid phone numbers moved to garbage file: 11
🚀 Ready for next cleaning step (BIC or IBAN)!


### ✅ Step: BIC (Bank Identifier Code) Validation & Cleanup
📌 Goal:
Check that all BIC values follow correct format:
8 or 11 characters, uppercase, letters and/or digits.
Invalid BICs ➡️ moved to garbage file AND replaced with blank in the clean dataset (NO row deletions!).

###  Final Consolidation Code Example:

In [41]:
import pandas as pd

# Load last cleaned dataset
final_clean_file = r'C:\Users\Andre\OneDrive\Documents\Belgium-Bank-Dataset\output\belgium_bank_clean_data_phones_valid.csv'
df = pd.read_csv(final_clean_file, dtype=str)

print("✅ Loaded dataset for finalization:", df.shape)

# Drop BIC (based on previous decision)
if 'bic' in df.columns:
    df.drop(columns=['bic'], inplace=True)
    print("🗑️ Dropped 'bic' column.")

# Final formatting check (strip whitespace)
for col in df.columns:
    df[col] = df[col].fillna('').astype(str).str.strip()

# Save final clean file
final_output_path = r'C:\Users\Andre\OneDrive\Documents\Belgium-Bank-Dataset\output\belgium_bank_clean_data_final.csv'
df.to_csv(final_output_path, index=False)
print(f"💾 Final clean dataset saved to: {final_output_path}")

# (Optional) Combine all garbage files into one
garbage_files = [
    r'C:\Users\Andre\OneDrive\Documents\Belgium-Bank-Dataset\output\belgium_bank_garbage_data.csv',
    # Add paths to other specific garbage datasets if not yet combined
]
garbage_dfs = [pd.read_csv(file, dtype=str) for file in garbage_files]
combined_garbage = pd.concat(garbage_dfs, ignore_index=True).drop_duplicates()

# Save combined garbage data
combined_garbage_path = r'C:\Users\Andre\OneDrive\Documents\Belgium-Bank-Dataset\output\belgium_bank_garbage_data.csv'
combined_garbage.to_csv(combined_garbage_path, index=False)
print(f"💾 All garbage data consolidated to: {combined_garbage_path}")


✅ Loaded dataset for finalization: (7428, 15)
🗑️ Dropped 'bic' column.
💾 Final clean dataset saved to: C:\Users\Andre\OneDrive\Documents\Belgium-Bank-Dataset\output\belgium_bank_clean_data_final.csv
💾 All garbage data consolidated to: C:\Users\Andre\OneDrive\Documents\Belgium-Bank-Dataset\output\belgium_bank_garbage_data.csv


### ✅ Here’s the code to execute this step properly:

In [42]:
# 📦 Imports
import pandas as pd

# ✅ Load final working dataset
file_path = r'C:\Users\Andre\OneDrive\Documents\Belgium-Bank-Dataset\output\belgium_bank_clean_data_final.csv'
df = pd.read_csv(file_path, dtype=str)

print("✅ Dataset loaded. Shape:", df.shape)

# ================================
# STEP 1: IBAN Validation Pattern
# ================================
iban_pattern = r'^BE\d{14}$'

# ================================
# STEP 2: Clean IBAN Column
# ================================
# Strip spaces and upper case
df['iban'] = df['iban'].fillna('').str.replace(' ', '').str.upper().str.strip()

# ================================
# STEP 3: Find Invalid IBANs
# ================================
invalid_iban_mask = ~df['iban'].str.match(iban_pattern, na=False)

invalid_ibans = df.loc[invalid_iban_mask, ['iban']].copy()
print(f"⚠️ Found {len(invalid_ibans)} invalid IBAN(s).")
print("🔍 Sample of invalid IBANs:")
print(invalid_ibans.head())

# ================================
# STEP 4: Save Invalid IBANs to Garbage
# ================================
# Load current garbage data to append invalid IBANs
garbage_path = r'C:\Users\Andre\OneDrive\Documents\Belgium-Bank-Dataset\output\belgium_bank_garbage_data.csv'
try:
    garbage_df = pd.read_csv(garbage_path, dtype=str)
except FileNotFoundError:
    garbage_df = pd.DataFrame()  # If no file exists yet

# Append only the invalid IBAN rows with full context
invalid_iban_full_rows = df.loc[invalid_iban_mask].copy()
garbage_df = pd.concat([garbage_df, invalid_iban_full_rows], ignore_index=True)

# Drop the invalid IBAN rows from the clean dataset
df_clean_final = df.loc[~invalid_iban_mask].copy()

# ================================
# STEP 5: Save Updated Files
# ================================
# Save updated clean dataset (OVERWRITE final clean file)
df_clean_final.to_csv(file_path, index=False)
print(f"💾 Updated clean dataset saved to: {file_path}")

# Save updated garbage file
garbage_df.to_csv(garbage_path, index=False)
print(f"💾 Invalid IBAN rows added to garbage file: {garbage_path}")

# ================================
# STEP 6: Final Summary
# ================================
print("\n📊 IBAN Cleaning Summary:")
print("✅ Clean dataset shape:", df_clean_final.shape)
print("🗑️ Garbage dataset shape:", garbage_df.shape)
print("🚀 Ready for ingestion with valid IBANs only!")



✅ Dataset loaded. Shape: (7428, 14)
⚠️ Found 29 invalid IBAN(s).
🔍 Sample of invalid IBANs:
                   iban
545     BE3773740389328
803     BE3406345244590
884     BE4063453276789
1574  BBE70464520761125
1901    BE9293057720614
💾 Updated clean dataset saved to: C:\Users\Andre\OneDrive\Documents\Belgium-Bank-Dataset\output\belgium_bank_clean_data_final.csv
💾 Invalid IBAN rows added to garbage file: C:\Users\Andre\OneDrive\Documents\Belgium-Bank-Dataset\output\belgium_bank_garbage_data.csv

📊 IBAN Cleaning Summary:
✅ Clean dataset shape: (7399, 14)
🗑️ Garbage dataset shape: (7018, 15)
🚀 Ready for ingestion with valid IBANs only!


### ✅ Step 1: Python Fix for Scientific Notation in account_number
Here is fully prepared code to fix it in-place:

In [43]:
import pandas as pd

# 🔹 Load the dataset from correct file path
file_path = r'C:\Users\Andre\OneDrive\Documents\Belgium-Bank-Dataset\output\belgium_bank_clean_data_final2.csv'
df = pd.read_csv(file_path, dtype=str)  # Read all columns as string to avoid numeric auto-formatting

print("✅ Dataset loaded. Shape:", df.shape)

# 🔹 Function to convert scientific notation to plain number
def fix_scientific_notation(value):
    try:
        if 'E' in str(value).upper():
            # Convert to float and format without scientific notation
            return '{0:.0f}'.format(float(value))
        else:
            return value  # Leave as is if not scientific notation
    except:
        return value  # Leave as is if conversion fails

# 🔹 Apply function to 'account_number' column
df['account_number'] = df['account_number'].apply(fix_scientific_notation)

print("✅ Scientific notation fixed in 'account_number'.")
print(df['account_number'].head(10))  # Preview for verification

# 🔹 Save back to same file for Data Wrangler / stakeholder use
df.to_csv(file_path, index=False)
print(f"💾 Cleaned data saved back to: {file_path}")


✅ Dataset loaded. Shape: (7399, 11)
✅ Scientific notation fixed in 'account_number'.
0      1462596514
1    860000000000
2    737000000000
3    733000000000
4     63461819055
5     63456572365
6    310000000000
7             NaN
8    980000000000
9      1119838728
Name: account_number, dtype: object
💾 Cleaned data saved back to: C:\Users\Andre\OneDrive\Documents\Belgium-Bank-Dataset\output\belgium_bank_clean_data_final2.csv


### ✅ Optional Refinement (If you want more explicit placeholders):
Instead of "", you could use clear markers like 'Unknown', '0000', or 'N/A' for better transparency:

In [44]:
custom_fill = {
    'last_name': 'Unknown',
    'initials': '',
    'street_name': 'Unknown',
    'house_number': '0',
    'postal_code': '0000',
    'city': 'Unknown',
    'date_of_birth': '',
    'account_number': 'Unknown',
    'iban': 'Unknown',
    'email': ''
}
df = df.fillna(custom_fill)


In [46]:
import pandas as pd

# === Load final cleaned dataset ===
final_path = r'C:\Users\Andre\OneDrive\Documents\Belgium-Bank-Dataset\output\belgium_bank_clean_data_final2.csv'
df = pd.read_csv(final_path, dtype=str)  # Load as string to avoid formatting issues

print("✅ Dataset loaded. Shape:", df.shape)
print("\n🔑 Columns and Data Types:\n", df.dtypes)
print("\n📊 First 5 records:\n", df.head())

# === 1. Check for Missing Values ===
missing_summary = df.isnull().sum()
print("\n🧹 Missing Values per Column:\n", missing_summary)
print("\n✅ Total Missing Values (should be 0 if fully cleaned):", missing_summary.sum())

# === 2. Unique & Distinct Values per Column ===
print("\n🔍 Distinct Counts per Column:")
for col in df.columns:
    print(f"{col}: {df[col].nunique()} unique values")

# === 3. Duplicates Check on Key Fields (IBAN, Account Number) ===
print("\n🔎 Checking for duplicates on 'iban' and 'account_number'...")
duplicate_iban = df[df.duplicated(subset=['iban'], keep=False)]
duplicate_account = df[df.duplicated(subset=['account_number'], keep=False)]

print(f"⚠️ Duplicate IBAN count: {len(duplicate_iban)}")
print(f"⚠️ Duplicate Account Number count: {len(duplicate_account)}")

# Optional: Export duplicates for review
duplicate_iban.to_csv(r'C:\Users\Andre\OneDrive\Documents\Belgium-Bank-Dataset\output\duplicates_iban.csv', index=False)
duplicate_account.to_csv(r'C:\Users\Andre\OneDrive\Documents\Belgium-Bank-Dataset\output\duplicates_account_number.csv', index=False)

# === 4. Statistical Overview for Numeric/Important Fields ===
print("\n📊 Quick Stats for House Number, Postal Code:")
print(df[['house_number', 'postal_code']].describe())

# === 5. Review Date of Birth Formatting ===
print("\n🗓️ Sample of Date of Birth values:")
print(df['date_of_birth'].dropna().head(10))  # Show samples of non-empty

# === 6. Example of Cleaned Records for Presentation ===
print("\n✅ Example of cleaned rows:")
print(df.sample(5, random_state=42))  # Random clean sample for review

print("\n🚀 Dataset ready for stakeholder handoff and ingestion!")


✅ Dataset loaded. Shape: (7399, 11)

🔑 Columns and Data Types:
 last_name         object
initials          object
street_name       object
house_number      object
postal_code       object
city              object
phone_number      object
date_of_birth     object
account_number    object
iban              object
email             object
dtype: object

📊 First 5 records:
      last_name initials               street_name house_number postal_code  \
0       Jacobs       C.              Gasmeterlaan          247        9000   
1        Dries       K.              Groningenlei           15        2550   
2    Vermeulen     KURT            Schongaustraat            7        9100   
3     Oirschot       L.            Ertbrandstraat          187        2950   
4  Compernolle       R.  Albrecht rodenbachstraat            4        8730   

                   city phone_number date_of_birth account_number  \
0                  Gent  475644230.0    24/12/1966     1462596514   
1               Kon

In [48]:
import pandas as pd

def clean_data(df):
    # ✅ Drop duplicate rows across all columns
    df = df.drop_duplicates()

    # ✅ Replace missing values with blanks for all key fields
    df = df.fillna({
        'last_name': "",
        'initials': "",
        'street_name': "",
        'house_number': "",
        'postal_code': "",
        'city': "",
        'date_of_birth': "",
        'account_number': "",
        'iban': "",
        'email': ""
    })

    # ✅ Drop rows with missing phone numbers (critical identifier)
    df = df.dropna(subset=['phone_number'])

    # ✅ Trim leading and trailing whitespace from key text fields
    columns_to_trim = ['last_name', 'initials', 'street_name', 'house_number',
                       'postal_code', 'city', 'date_of_birth', 'account_number',
                       'iban', 'email']
    for col in columns_to_trim:
        df[col] = df[col].str.strip()

    # ✅ Capitalize important text columns for consistency
    df['last_name'] = df['last_name'].str.capitalize()
    df['street_name'] = df['street_name'].str.capitalize()
    df['city'] = df['city'].str.capitalize()

    return df

# ✅ Load dataset safely (without pyarrow)
df = pd.read_csv(r'C:\Users\Andre\OneDrive\Documents\Belgium-Bank-Dataset\output\belgium_bank_clean_data_final2.csv', dtype=str)

# ✅ Apply cleaning process
df_clean = clean_data(df.copy())

# ✅ Save cleaned dataset back to same file path (overwrite old version)
final_output_path = r'C:\Users\Andre\OneDrive\Documents\Belgium-Bank-Dataset\output\belgium_bank_clean_data_final2.csv'
df_clean.to_csv(final_output_path, index=False)

# ✅ Final confirmations
print(f"🚀 Final cleaned dataset saved successfully at: {final_output_path}")
print(f"✅ Final Dataset Shape: {df_clean.shape}")
print("\n📊 Sample of final cleaned data:\n", df_clean.head())



🚀 Final cleaned dataset saved successfully at: C:\Users\Andre\OneDrive\Documents\Belgium-Bank-Dataset\output\belgium_bank_clean_data_final2.csv
✅ Final Dataset Shape: (6788, 11)

📊 Sample of final cleaned data:
      last_name initials               street_name house_number postal_code  \
0       Jacobs       C.              Gasmeterlaan          247        9000   
1        Dries       K.              Groningenlei           15        2550   
2    Vermeulen     KURT            Schongaustraat            7        9100   
3     Oirschot       L.            Ertbrandstraat          187        2950   
4  Compernolle       R.  Albrecht rodenbachstraat            4        8730   

                   city phone_number date_of_birth account_number  \
0                  Gent  475644230.0    24/12/1966     1462596514   
1               Kontich   32899040.0                 860000000000   
2          Sint-niklaas   32966602.0    13/08/1965   737000000000   
3  Kapellen (antwerpen)   36052791.0    25/

In [49]:
df_clean.isnull().sum()
df_clean.nunique()
df_clean.dtypes     

# Duplicates in IBAN
duplicate_iban = df_clean[df_clean.duplicated('iban', keep=False)]
print(f"⚠️ Duplicate IBAN count: {len(duplicate_iban)}")

# Duplicates in account_number
duplicate_account = df_clean[df_clean.duplicated('account_number', keep=False)]
print(f"⚠️ Duplicate Account Number count: {len(duplicate_account)}")



⚠️ Duplicate IBAN count: 257
⚠️ Duplicate Account Number count: 4284


In [50]:
import pandas as pd

# ✅ Load the current cleaned dataset
file_path = r'C:\Users\Andre\OneDrive\Documents\Belgium-Bank-Dataset\output\belgium_bank_clean_data_final2.csv'
df = pd.read_csv(file_path, dtype=str)
print("✅ Dataset loaded. Shape:", df.shape)

# =============================
# Step 1: Detect Duplicates
# =============================

# Find duplicate IBAN rows (excluding first occurrence)
duplicate_iban = df[df.duplicated('iban', keep='first')]
print(f"⚠️ Duplicate IBANs detected: {len(duplicate_iban)}")

# Find duplicate Account Number rows (excluding first occurrence)
duplicate_account = df[df.duplicated('account_number', keep='first')]
print(f"⚠️ Duplicate Account Numbers detected: {len(duplicate_account)}")

# =============================
# Step 2: Combine duplicates
# =============================
# Combine all detected duplicates into one garbage dataset
garbage_data = pd.concat([duplicate_iban, duplicate_account]).drop_duplicates()
print(f"🗑️ Total rows to move to garbage file: {len(garbage_data)}")

# =============================
# Step 3: Remove duplicates from clean dataset
# =============================
# Remove all these duplicates from original dataframe
df_clean = df.drop(garbage_data.index).reset_index(drop=True)
print(f"✅ Final cleaned dataset shape (after removing dups): {df_clean.shape}")

# =============================
# Step 4: Save cleaned and garbage data
# =============================

# Save clean dataset back for final review
clean_path = r'C:\Users\Andre\OneDrive\Documents\Belgium-Bank-Dataset\output\belgium_bank_clean_data_final_review.csv'
df_clean.to_csv(clean_path, index=False)
print(f"💾 Final clean dataset saved at: {clean_path}")

# Save garbage (duplicates) data
garbage_path = r'C:\Users\Andre\OneDrive\Documents\Belgium-Bank-Dataset\output\belgium_bank_garbage_data.csv'
garbage_data.to_csv(garbage_path, index=False)
print(f"💾 Garbage data (duplicates) saved at: {garbage_path}")

# =============================
# Step 5: Final Summary
# =============================
print("\n📊 Cleaning Summary:")
print(f"Original dataset shape: {df.shape}")
print(f"Duplicates moved to garbage: {len(garbage_data)}")
print(f"Final dataset shape: {df_clean.shape}")


✅ Dataset loaded. Shape: (6788, 11)
⚠️ Duplicate IBANs detected: 132
⚠️ Duplicate Account Numbers detected: 4077
🗑️ Total rows to move to garbage file: 4085
✅ Final cleaned dataset shape (after removing dups): (2703, 11)
💾 Final clean dataset saved at: C:\Users\Andre\OneDrive\Documents\Belgium-Bank-Dataset\output\belgium_bank_clean_data_final_review.csv
💾 Garbage data (duplicates) saved at: C:\Users\Andre\OneDrive\Documents\Belgium-Bank-Dataset\output\belgium_bank_garbage_data.csv

📊 Cleaning Summary:
Original dataset shape: (6788, 11)
Duplicates moved to garbage: 4085
Final dataset shape: (2703, 11)


In [52]:
import pandas as pd

# Load the final cleaned dataset
df = pd.read_csv(r'C:\Users\Andre\OneDrive\Documents\Belgium-Bank-Dataset\output\belgium_bank_clean_data_final2.csv', dtype=str)

# ✅ Fix phone_number by removing '.0' and ensuring it's a string
df['phone_number'] = df['phone_number'].astype(str).str.replace(r'\.0$', '', regex=True)

# ✅ Save the fixed dataset back to the same location (overwrite)
df.to_csv(r'C:\Users\Andre\OneDrive\Documents\Belgium-Bank-Dataset\output\belgium_bank_clean_data_final2.csv', index=False)

print("🚀 Final phone number fix applied. Dataset is now fully clean and saved.")
print(df['phone_number'].head(10))  # Quick check
# Load the dataset again for verification
df = pd.read_csv(r'C:\Users\Andre\OneDrive\Documents\Belgium-Bank-Dataset\output\belgium_bank_clean_data_final2.csv', dtype=str)

# Preview first 10 rows of phone_number to verify fix
print(df['phone_number'].head(10))


🚀 Final phone number fix applied. Dataset is now fully clean and saved.
0    475644230
1     32899040
2     32966602
3     36052791
4     50781034
5    472453099
6    475391350
7    476075682
8    476385716
9    478419735
Name: phone_number, dtype: object
0    475644230
1     32899040
2     32966602
3     36052791
4     50781034
5    472453099
6    475391350
7    476075682
8    476385716
9    478419735
Name: phone_number, dtype: object
